# Additional real dataset results:

In [19]:
import numpy as np
import pandas as pd
import os

from nonconformist.cp import IcpRegressor
from nonconformist.nc import NcFactory
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-white")

from clover.locart import LocalRegressionSplit, LocartSplit, MondrianRegressionSplit, RegressionSplit
from acpi import ACPI
from clover.scores import RegressionScore, LocalRegressionScore
import time

from clover.locart import LocartSplit, MondrianRegressionSplit
from clover.scores import RegressionScore
from clover.utils import compute_interval_length, split, smis
import gc

original_path = os.getcwd()

# figure path
images_dir = "figures"

## Defining function to run big dataset experiment:

In [20]:
def obtain_main_metrics_real(
    data_name,
    is_fitted = True,
    figname_tree = "tree",
    base_model = RandomForestRegressor,
    sig = 0.1, 
    test_size = 0.2,
    calib_size = 0.5, 
    random_seed = 1250,
    split_calib = False,
    plot_tree = False,
    nbins = 30,
    criterion = "squared_error",
    max_depth = None,
    max_leaf_nodes = None,
    min_samples_leaf = 150,
    prune = True,
    random_projections=False,
    h=20,
    m=300,
    n_estimators = 100,
    **kwargs):

    # importing data, selecting some rows and then splitting
    data_path = os.path.normpath(os.getcwd() + os.sep + os.pardir) + "/data/processed/" + data_name + ".csv"

    # reading data using pandas
    data = pd.read_csv(data_path)
    y = data["target"].to_numpy()
    X = (data.
    drop("target", axis = 1).
    to_numpy())

    print("Number of samples used for training and calibration: {}".format((1 - test_size)*X.shape[0]))
    print("Number of samples used for testing: {}".format(test_size*X.shape[0]))

    # splitting data into train, calibration and test
    data = split(X, y, test_size, calib_size, calibrate = True, random_seed = random_seed)

    if data_name == "amazon":
        X_train = data["X_train"].flatten()
        X_test = data["X_test"].flatten()
        X_calib = data["X_calib"].flatten()

        tfidf = TfidfVectorizer(max_features=500)
        X_train = tfidf.fit_transform(X_train).toarray()
        X_test = tfidf.transform(X_test).toarray()
        X_calib = tfidf.transform(X_calib).toarray()
        features = tfidf.get_feature_names_out()
        np.savetxt(f"data/processed/amazon_features_test", features, fmt="%s")

        data["X_train"] = X_train
        data["X_test"] = X_test
        data["X_calib"] = X_calib

    # setting seed
    np.random.seed(random_seed)


    model = base_model(**kwargs).fit(data["X_train"], data["y_train"])

    # fitting mondrian regression split
    print("Fitting mondrian regression split")
    start_mondrian_split = time.time()
    micp = MondrianRegressionSplit(model, is_fitted = is_fitted, alpha = sig, k = nbins, **kwargs)
    micp.fit(data["X_train"], data["y_train"])
    micp.calibrate(data["X_calib"], data["y_calib"])

    end_mondrian_split = time.time() - start_mondrian_split
    print("Time Elapsed to fit mondrian regression split: ", end_mondrian_split)

    print("Computing metrics")
    start_mondrian_split = time.time()
    # predictions for empirical metrics
    micp_pred = micp.predict(data["X_test"])

    # smis
    micp_smis = smis(micp_pred, data["y_test"], alpha = sig)
        

    # mean interval length
    micp_interval_len = np.mean(compute_interval_length(micp_pred))


    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= micp_pred[:, 0], 
        data["y_test"] <= micp_pred[:, 1]) + 0
    micp_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    micp_interval_len_cover = np.mean(compute_interval_length(micp_pred[cover_idx]))


    end_mondrian_split = time.time() - start_mondrian_split
    print("Time Elapsed to compute statistics for mondrian regression split: ", end_mondrian_split)

    # deleting mondrian objects
    del micp
    del micp_pred
    del cover_idx
    del marg_cover
    gc.collect()

    print("Fitting locart")
    start_loc = time.time()
    # fitting locart
    locart_obj = LocartSplit(nc_score = RegressionScore, base_model = model, is_fitted = is_fitted, 
                             alpha = sig, split_calib = split_calib, **kwargs)
    locart_obj.fit(data["X_train"], data["y_train"])
    locart_obj.calib(data["X_calib"], data["y_calib"], max_depth = max_depth, 
    max_leaf_nodes = max_leaf_nodes, min_samples_leaf = min_samples_leaf, criterion = criterion, prune_tree = prune)
    
    end_loc = time.time() - start_loc
    print("Time Elapsed to fit Locart: ", end_loc)

    print("Computing metrics")
    start_loc = time.time()
    # predictions
    locart_pred = np.array(locart_obj.predict(data["X_test"]))

    # smis
    locart_smis = smis(locart_pred, data["y_test"], alpha = sig)

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= locart_pred[:, 0], 
        data["y_test"] <= locart_pred[:, 1]) + 0
    locart_ave_marginal_cov = np.mean(
        marg_cover
    )

    # mean interval length
    locart_interval_len = np.mean(compute_interval_length(locart_pred))

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    locart_interval_len_cover = np.mean(compute_interval_length(locart_pred[cover_idx]))
    end_loc = time.time() - start_loc
    print("Time Elapsed to compute metrics for Locart: ", end_loc)

    # deleting locart objects
    del locart_obj
    del locart_pred
    del cover_idx
    del marg_cover
    gc.collect()

    # fitting normal difficulty locart
    print("Fitting A-locart to toy example:")
    start_loc = time.time()
    dlocart_obj = LocartSplit(nc_score = RegressionScore, cart_type = "CART", base_model = model, is_fitted = is_fitted,
                               alpha = sig, split_calib = split_calib, 
                              weighting = True, **kwargs)
    dlocart_obj.fit(data["X_train"], data["y_train"])
    dlocart_obj.calib(data["X_calib"], data["y_calib"], max_depth = max_depth, 
    max_leaf_nodes = max_leaf_nodes, min_samples_leaf = min_samples_leaf, criterion = criterion, prune_tree = prune)
    
    end_loc = time.time() - start_loc
    print("Time Elapsed to fit A-Locart: ", end_loc)

    print("Computing metrics")
    start_loc = time.time()
    # predictions
    dlocart_pred = np.array(dlocart_obj.predict(data["X_test"]))

    # smis
    dlocart_smis = smis(dlocart_pred, data["y_test"], alpha = sig)

    # mean interval length
    dlocart_interval_len = np.mean(compute_interval_length(dlocart_pred))

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= dlocart_pred[:, 0], 
        data["y_test"] <= dlocart_pred[:, 1]) + 0
    dlocart_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    dlocart_interval_len_cover = np.mean(compute_interval_length(dlocart_pred[cover_idx]))

    end_loc = time.time() - start_loc
    print("Time Elapsed to compute metrics for A-Locart: ", end_loc)
    # deleting alocart objects
    del dlocart_obj
    del dlocart_pred
    del cover_idx
    del marg_cover
    gc.collect()

    # fitting normal loforest 
    print("Fitting loforest")
    start_loc = time.time()
    # fitting loforest
    loforest_obj = LocartSplit(nc_score = RegressionScore, cart_type = "RF", base_model = model, is_fitted = is_fitted, 
                             alpha = sig, split_calib = split_calib, **kwargs)
    loforest_obj.fit(data["X_train"], data["y_train"])
    loforest_obj.calib(data["X_calib"], data["y_calib"], max_depth = max_depth, 
    max_leaf_nodes = max_leaf_nodes, min_samples_leaf = min_samples_leaf, criterion = criterion, prune_tree = prune, n_estimators = n_estimators)
    
    end_loc = time.time() - start_loc
    print("Time Elapsed to fit Loforest: ", end_loc)

    print("Computing metrics")
    start_loc = time.time()
    # predictions
    loforest_pred = np.array(loforest_obj.predict(data["X_test"]))

    # smis
    loforest_smis = smis(loforest_pred, data["y_test"], alpha = sig)

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= loforest_pred[:, 0], 
        data["y_test"] <= loforest_pred[:, 1]) + 0
    loforest_ave_marginal_cov = np.mean(
        marg_cover
    )

    # mean interval length
    loforest_interval_len = np.mean(compute_interval_length(loforest_pred))

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    loforest_interval_len_cover = np.mean(compute_interval_length(loforest_pred[cover_idx]))
    end_loc = time.time() - start_loc
    print("Time Elapsed to compute metrics for Loforest: ", end_loc)

    # deleting loforest objects
    del loforest_obj
    del loforest_pred
    del cover_idx
    del marg_cover
    gc.collect()

    print("Fitting A-Loforest")
    start_loc = time.time()
    # fitting locart
    aloforest_obj = LocartSplit(nc_score = RegressionScore, cart_type = "RF", base_model = model, is_fitted = is_fitted, 
                             alpha = sig, split_calib = split_calib, weighting = True, **kwargs)
    aloforest_obj.fit(data["X_train"], data["y_train"])
    aloforest_obj.calib(data["X_calib"], data["y_calib"], max_depth = max_depth, 
    max_leaf_nodes = max_leaf_nodes, min_samples_leaf = min_samples_leaf, criterion = criterion, prune_tree = prune, n_estimators = n_estimators)
    
    end_loc = time.time() - start_loc
    print("Time Elapsed to fit A-Loforest: ", end_loc)

    print("Computing metrics")
    start_loc = time.time()
    # predictions
    aloforest_pred = np.array(aloforest_obj.predict(data["X_test"]))

    # smis
    aloforest_smis = smis(aloforest_pred, data["y_test"], alpha = sig)

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= aloforest_pred[:, 0], 
        data["y_test"] <= aloforest_pred[:, 1]) + 0
    aloforest_ave_marginal_cov = np.mean(
        marg_cover
    )

    # mean interval length
    aloforest_interval_len = np.mean(compute_interval_length(aloforest_pred))

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    aloforest_interval_len_cover = np.mean(compute_interval_length(aloforest_pred[cover_idx]))
    end_loc = time.time() - start_loc
    print("Time Elapsed to compute metrics for A-Loforest: ", end_loc)

    # deleting a-loforest objects
    del aloforest_obj
    del aloforest_pred
    del cover_idx
    del marg_cover
    gc.collect()


    print("Fitting regression split")
    start_split = time.time()
    icp = RegressionSplit(base_model = model, alpha = sig, is_fitted = True)
    icp.fit(data["X_train"], data["y_train"])
    icp.calibrate(data["X_calib"], data["y_calib"])

    end_split = time.time() - start_split
    print("Time Elapsed to fit regression split: ", end_split)

    print("Computing metrics")
    start_split = time.time()
    # predictions
    icp_pred = icp.predict(data["X_test"])

    # smis
    icp_smis = smis(icp_pred, data["y_test"], alpha = sig)

    # ICP interval length
    icp_interval_len = np.mean(compute_interval_length(icp_pred))

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= icp_pred[:, 0], 
        data["y_test"] <= icp_pred[:, 1]) + 0
    icp_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    icp_interval_len_cover = np.mean(compute_interval_length(icp_pred[cover_idx]))

    end_split = time.time() - start_split
    print("Time Elapsed to compute statistics for regression split: ", end_split)

    # deleting ICP objects
    del icp
    del icp_pred
    del cover_idx
    del marg_cover
    gc.collect()

    # fitting wighted regression split
    print("Fitting weighted regression split")
    start_weighted_split = time.time()
    wicp = LocalRegressionSplit(model, is_fitted = is_fitted, alpha = sig, **kwargs)
    wicp.fit(data["X_train"], data["y_train"])
    wicp.calibrate(data["X_calib"], data["y_calib"])

    end_weighted_split = time.time() - start_weighted_split
    print("Time Elapsed to fit weighted regression split: ", end_weighted_split)

    print("Computing metrics")
    start_weighted_split = time.time()
    # predictions
    wicp_pred = wicp.predict(data["X_test"])

    # smis
    wicp_smis = smis(wicp_pred, data["y_test"], alpha = sig)

    # ICP interval length
    wicp_interval_len = np.mean(compute_interval_length(wicp_pred))

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= wicp_pred[:, 0], 
        data["y_test"] <= wicp_pred[:, 1]) + 0
    wicp_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    wicp_interval_len_cover = np.mean(compute_interval_length(wicp_pred[cover_idx]))
    end_weighted_split = time.time() - start_weighted_split
    print("Time Elapsed to compute statistics for weighted regression split: ", end_weighted_split)

    # deleting WICP objects
    del wicp
    del wicp_pred
    del cover_idx
    del marg_cover
    gc.collect()


    start_loc = time.time()

    print("Fitting W-LOFOREST")
    wlocart_obj = LocartSplit(
        nc_score=LocalRegressionScore,
        cart_type="RF",
        base_model=model,
        is_fitted=is_fitted,
        alpha=sig,
        split_calib=split_calib,
        **kwargs,
    )
    wlocart_obj.fit(data["X_train"], data["y_train"])
    wlocart_obj.calib(
        data["X_calib"],
        data["y_calib"],
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_nodes,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        prune_tree=prune,
        random_projections=random_projections,
        n_estimators = n_estimators,
        m=m,
        h=h,
    )

    end_weighted_split = time.time() - start_weighted_split
    print("Time Elapsed to fit W-LOFOREST: ", end_weighted_split)

    print("Computing metrics")
    start_weighted_split = time.time()
    # predictions
    wlocart_pred = wlocart_obj.predict(data["X_test"])

    # smis
    wlocart_smis = smis(wlocart_pred, data["y_test"], alpha = sig)

    # ICP interval length
    wlocart_interval_len = np.mean(compute_interval_length(wlocart_pred))

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= wlocart_pred[:, 0], 
        data["y_test"] <= wlocart_pred[:, 1]) + 0
    wlocart_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    wlocart_interval_len_cover = np.mean(compute_interval_length(wlocart_pred[cover_idx]))
    end_weighted_split = time.time() - start_weighted_split
    print("Time Elapsed to compute statistics for W-LOFOREST: ", end_weighted_split)

    # deleting WICP objects
    del wlocart_obj
    del wlocart_pred
    del cover_idx
    del marg_cover
    gc.collect()



    print("Fitting ACPI")
    start_weighted_split = time.time()
    acpi = ACPI(model_cali = model, n_estimators = 100)
    acpi.fit(data["X_test"], data["y_test"], nonconformity_func = None)
    acpi.fit_calibration(data["X_calib"], data["y_calib"], quantile = 1 - sig, only_qrf = True)

    end_weighted_split = time.time() - start_weighted_split
    print("Time Elapsed to fit ACPI: ", end_weighted_split)

    print("Computing metrics")
    start_weighted_split = time.time()
    # predictions
    acpi_pred = np.stack((acpi.predict_pi(data["X_test"], method="qrf")), axis=-1)

    # smis
    acpi_smis = smis(acpi_pred, data["y_test"], alpha = sig)

    # ICP interval length
    acpi_interval_len = np.mean(compute_interval_length(acpi_pred))

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= acpi_pred[:, 0], 
        data["y_test"] <= acpi_pred[:, 1]) + 0
    acpi_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    acpi_interval_len_cover = np.mean(compute_interval_length(acpi_pred[cover_idx]))
    end_weighted_split = time.time() - start_weighted_split
    print("Time Elapsed to compute statistics for ACPI: ", end_weighted_split)

    # deleting ACPI objects
    del acpi
    del acpi_pred
    del cover_idx
    del marg_cover
    gc.collect()
    
    all_results =  pd.DataFrame(data = {"Methods":["LOCART", "A-LOCART", "LOFOREST", "A-LOFOREST",
                                                   "Regresion split", "Weighted regression split", 
                                                   "Mondrian regression split", "ACPI", "W-LOFOREST"],
    "Average marginal coverage":[locart_ave_marginal_cov, dlocart_ave_marginal_cov, loforest_ave_marginal_cov, aloforest_ave_marginal_cov, 
                                 icp_ave_marginal_cov, wicp_ave_marginal_cov, micp_ave_marginal_cov, acpi_ave_marginal_cov,
                                 wlocart_ave_marginal_cov],
    "Average interval length":[locart_interval_len, dlocart_interval_len, loforest_interval_len, aloforest_interval_len,
                               icp_interval_len, wicp_interval_len, micp_interval_len, acpi_interval_len,
                               wlocart_interval_len],
    "Average interval length given coverage":[locart_interval_len_cover, dlocart_interval_len_cover, loforest_interval_len_cover, aloforest_interval_len_cover,
                                              icp_interval_len_cover, wicp_interval_len_cover, micp_interval_len_cover, acpi_interval_len_cover,
                                              wlocart_interval_len_cover],
    "SMIS values":[locart_smis, dlocart_smis, loforest_smis, aloforest_smis, icp_smis, wicp_smis, micp_smis, acpi_smis, wlocart_smis]})
        
    # plotting tree to visualize partitions
    if plot_tree:
        plt.figure(figsize=(25, 20))
        locart_obj.plot_locart()
        plt.savefig(f"{images_dir}/{figname_tree}")
        plt.show()
    
    return all_results

### Running kernel dataset:

In [22]:
kernel_results = obtain_main_metrics_real(data_name = "kernel", random_seed = 30, random_state = 45, min_samples_leaf = 150, n_estimators = 100)

Number of samples used for training and calibration: 193280.0
Number of samples used for testing: 48320.0
Fitting mondrian regression split
Time Elapsed to fit mondrian regression split:  8.87926197052002
Computing metrics
Time Elapsed to compute statistics for mondrian regression split:  2.7890756130218506
Fitting locart
Time Elapsed to fit Locart:  24.89926505088806
Computing metrics
Time Elapsed to compute metrics for Locart:  1.228313684463501
Fitting A-locart to toy example:
Time Elapsed to fit A-Locart:  37.11011481285095
Computing metrics
Time Elapsed to compute metrics for A-Locart:  2.614086151123047
Fitting loforest
Time Elapsed to fit Loforest:  21.565516233444214
Computing metrics
Time Elapsed to compute metrics for Loforest:  1.9205327033996582
Fitting A-Loforest
Time Elapsed to fit A-Loforest:  29.977654218673706
Computing metrics
Time Elapsed to compute metrics for A-Loforest:  3.3845417499542236
Fitting regression split
Time Elapsed to fit regression split:  2.385746955

100%|██████████| 50/50 [19:07<00:00, 22.95s/it]


Time Elapsed to fit ACPI:  1197.154717206955
Computing metrics
Time Elapsed to compute statistics for ACPI:  2.2162208557128906


In [23]:
kernel_results

,Methods,Average marginal coverage,Average interval length,Average interval length given coverage,SMIS values
0,LOCART,0.901283,4.292009,4.311993,-6.411761
1,A-LOCART,0.901987,4.019274,4.024302,-6.146922
2,LOFOREST,0.912790,4.291809,4.262284,-6.416687
3,A-LOFOREST,0.911341,4.029497,3.994142,-6.040283
4,Regresion split,0.902732,4.228260,4.228260,-10.465670
5,Weighted regression split,0.899379,3.961186,4.023055,-5.501266
6,Mondrian regression split,0.901966,4.152145,4.153313,-6.535449
7,ACPI,0.921896,4.202459,4.110069,-5.962586
8,W-LOFOREST,0.905008,4.098669,4.190425,-5.474092


## Running amazon dataset